<a href="https://colab.research.google.com/github/kumuds4/BCH/blob/master/Making_the_Most_of_your_Colab_Subscription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making the Most of your Colab Subscription



## Faster GPUs

Users who have purchased one of Colab's paid plans have access to faster GPUs and more memory. You can upgrade your notebook's GPU settings in `Runtime > Change runtime type` in the menu to select from several accelerator options, subject to availability.

The free of charge version of Colab grants access to Nvidia's T4 GPUs subject to quota restrictions and availability.

You can see what GPU you've been assigned at any time by executing the following cell. If the execution result of running the code cell below is "Not connected to a GPU", you can change the runtime by going to `Runtime > Change runtime type` in the menu to enable a GPU accelerator, and then re-execute the code cell.


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In order to use a GPU with your notebook, select the `Runtime > Change runtime type` menu, and then set the hardware accelerator to the desired option.

## More memory

Users who have purchased one of Colab's paid plans have access to high-memory VMs when they are available. More powerful GPUs are always offered with high-memory VMs.



You can see how much memory you have available at any time by running the following code cell. If the execution result of running the code cell below is "Not using a high-RAM runtime", then you can enable a high-RAM runtime via `Runtime > Change runtime type` in the menu. Then select High-RAM in the Runtime shape toggle button. After, re-execute the code cell.


In [ ]:
import psutil

ram_gb = psutil.virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

## Longer runtimes

All Colab runtimes are reset after some period of time (which is faster if the runtime isn't executing code). Colab Pro and Pro+ users have access to longer runtimes than those who use Colab free of charge.

## Background execution

Colab Pro+ users have access to background execution, where notebooks will continue executing even after you've closed a browser tab. This is always enabled in Pro+ runtimes as long as you have compute units available.



## Relaxing resource limits in Colab Pro

Your resources are not unlimited in Colab. To make the most of Colab, avoid using resources when you don't need them. For example, only use a GPU when required and close Colab tabs when finished.



If you encounter limitations, you can relax those limitations by purchasing more compute units via Pay As You Go. Anyone can purchase compute units via [Pay As You Go](https://colab.research.google.com/signup); no subscription is required.

## Send us feedback!

If you have any feedback for us, please let us know. The best way to send feedback is by using the Help > 'Send feedback...' menu. If you encounter usage limits in Colab Pro consider subscribing to Pro+.

If you encounter errors or other issues with billing (payments) for Colab Pro, Pro+, or Pay As You Go, please email [colab-billing@google.com](mailto:colab-billing@google.com).

## More Resources

### Working with Notebooks in Colab
- [Overview of Colab](/notebooks/basic_features_overview.ipynb)
- [Guide to Markdown](/notebooks/markdown_guide.ipynb)
- [Importing libraries and installing dependencies](/notebooks/snippets/importing_libraries.ipynb)
- [Saving and loading notebooks in GitHub](https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb)
- [Interactive forms](/notebooks/forms.ipynb)
- [Interactive widgets](/notebooks/widgets.ipynb)

<a name="working-with-data"></a>
### Working with Data
- [Loading data: Drive, Sheets, and Google Cloud Storage](/notebooks/io.ipynb)
- [Charts: visualizing data](/notebooks/charts.ipynb)
- [Getting started with BigQuery](/notebooks/bigquery.ipynb)

### Machine Learning Crash Course
These are a few of the notebooks from Google's online Machine Learning course. See the [full course website](https://developers.google.com/machine-learning/crash-course/) for more.
- [Intro to Pandas DataFrame](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/pandas_dataframe_ultraquick_tutorial.ipynb)
- [Linear regression with tf.keras using synthetic data](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/linear_regression_with_synthetic_data.ipynb)


<a name="using-accelerated-hardware"></a>
### Using Accelerated Hardware
- [TensorFlow with GPUs](/notebooks/gpu.ipynb)
- [TPUs in Colab](/notebooks/tpu.ipynb)

<a name="machine-learning-examples"></a>

## Machine Learning Examples

To see end-to-end examples of the interactive machine learning analyses that Colab makes possible, check out these tutorials using models from [TensorFlow Hub](https://tfhub.dev).

A few featured examples:

- [Retraining an Image Classifier](https://tensorflow.org/hub/tutorials/tf2_image_retraining): Build a Keras model on top of a pre-trained image classifier to distinguish flowers.
- [Text Classification](https://tensorflow.org/hub/tutorials/tf2_text_classification): Classify IMDB movie reviews as either *positive* or *negative*.
- [Style Transfer](https://tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization): Use deep learning to transfer style between images.
- [Multilingual Universal Sentence Encoder Q&A](https://tensorflow.org/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa): Use a machine learning model to answer questions from the SQuAD dataset.
- [Video Interpolation](https://tensorflow.org/hub/tutorials/tweening_conv3d): Predict what happened in a video between the first and the last frame.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import files
uploaded = files.upload()

Saving making_the_most_of_your_colab_subscription (14).py to making_the_most_of_your_colab_subscription (14).py


In [23]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import logging
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
#######################################################################
#latest version
# Logging Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s]: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Device Configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using Device: {DEVICE}")

# Polar Code Generator Class
class PolarCodeGenerator:
    def __init__(self, N, K, crc_type='CRC-7'):
        self.N = N
        self.K = K
        self.crc_type = crc_type
        self.crc_polynomials = {
            'CRC-7': {
                'polynomial': [1, 1, 1, 0, 0, 1, 1],
                'length': 7
            }
        }

    def generate_info_bits(self):
        return np.random.randint(2, size=self.K)

    def polar_encode(self, info_bits):
        codeword = np.zeros(self.N, dtype=int)
        codeword[:len(info_bits)] = info_bits
        return codeword

# Channel Simulator Class
class EnhancedChannelSimulator:
    def __init__(self, channel_type='AWGN'):
        self.channel_type = channel_type

    def simulate(self, encoded_signal, snr_db):
        # Convert bits {0,1} to BPSK: {+1, -1}
        bpsk_signal = 1 - 2 * encoded_signal

        # Convert SNR from dB to linear scale
        snr_linear = 10 ** (snr_db / 10)

        # Compute signal power
        signal_power = np.mean(bpsk_signal**2)

        # Noise power calculation
        noise_power = signal_power / snr_linear
        noise_std = np.sqrt(noise_power / 2.0)

        # Channel-specific simulation
        if self.channel_type == 'AWGN':
            noise = np.random.normal(0, noise_std, bpsk_signal.shape)
            received_signal = bpsk_signal + noise
        elif self.channel_type == 'Rayleigh':
            fading = np.random.rayleigh(scale=1.0, size=bpsk_signal.shape)
            noise = np.random.normal(0, noise_std, bpsk_signal.shape)
            received_signal = fading * bpsk_signal + noise
        else:
            raise ValueError(f"Unsupported channel type: {self.channel_type}")

        # Convert back to binary representation
        return (received_signal > 0).astype(float)

# Dataset Preparation Function
def prepare_polar_dataset(polar_code_gen, num_samples, snr_db=5, channel_type="AWGN"):
    channel_simulator = EnhancedChannelSimulator(channel_type=channel_type)

    X = []
    y = []

    for _ in range(num_samples):
        # Generate information bits
        info_bits = polar_code_gen.generate_info_bits()

        # Encode polar code
        encoded_signal = polar_code_gen.polar_encode(info_bits)

        # Channel simulation
        received_signal = channel_simulator.simulate(encoded_signal, snr_db)

        # Store features and labels
        X.append(received_signal)

        # Binary classification label
        y.append(1 if np.mean(info_bits) > 0.5 else 0)

    return np.array(X), np.array(y)

# Neural Network Decoder
class EnhancedRNNDecoder(nn.Module):
    def __init__(self, input_size, device=None):
        super(EnhancedRNNDecoder, self).__init__()

        self.device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.model = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        ).to(self.device)

    def forward(self, x):
        x = x.to(self.device)

        if x.dim() == 1:
            x = x.unsqueeze(0)

        if x.dim() > 2:
            x = x.view(x.size(0), -1)

        return self.model(x).squeeze(-1)

# Decoder Trainer
class DecoderTrainer:
    def __init__(self, model, learning_rate=1e-3):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = model.to(self.device)
        self.criterion = nn.BCEWithLogitsLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode='min', factor=0.5, patience=5
        )
        self.train_losses = []
        self.val_losses = []

    def train(self, X, y, epochs=50, batch_size=32, validation_split=0.2):
        # Ensure inputs are tensors
        if not isinstance(X, torch.Tensor):
            X = torch.FloatTensor(X)
        if not isinstance(y, torch.Tensor):
            y = torch.FloatTensor(y)

        # Move to device
        X = X.to(self.device)
        y = y.to(self.device)

        # Reshape y if needed
        y = y.view(-1, 1)

        # Split into train and validation
        train_size = int((1 - validation_split) * len(X))
        X_train, X_val = X[:train_size], X[train_size:]
        y_train, y_val = y[:train_size], y[train_size:]

        # Create data loaders
        train_dataset = TensorDataset(X_train, y_train)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        val_dataset = TensorDataset(X_val, y_val)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        # Training loop
        for epoch in range(epochs):
            # Training phase
            self.model.train()
            train_loss = self._train_epoch(train_loader)
            self.train_losses.append(train_loss)

            # Validation phase
            self.model.eval()
            val_loss = self._validate(val_loader)
            self.val_losses.append(val_loss)

            # Learning rate scheduling
            self.scheduler.step(val_loss)

            # Print progress
            print(f"Epoch [{epoch+1}/{epochs}], "
                  f"Train Loss: {train_loss:.4f}, "
                  f"Val Loss: {val_loss:.4f}")

        return self.train_losses, self.val_losses

    def _train_epoch(self, dataloader):
        total_loss = 0
        for batch_X, batch_y in dataloader:
            self.optimizer.zero_grad()
            outputs = self.model(batch_X)
            loss = self.criterion(outputs, batch_y)
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item()
        return total_loss / len(dataloader)

    def _validate(self, dataloader):
        total_loss = 0
        with torch.no_grad():
            for batch_X, batch_y in dataloader:
                outputs = self.model(batch_X)
                loss = self.criterion(outputs, batch_y)
                total_loss += loss.item()
        return total_loss / len(dataloader)

    def predict(self, X):
        self.model.eval()
        with torch.no_grad():
            X = torch.FloatTensor(X).to(self.device)
            outputs = torch.sigmoid(self.model(X))
        return outputs.cpu().numpy().flatten()

# Performance Metrics Function
def compute_performance_metrics(y_true, y_pred):
    y_true_binary = (y_true > 0.5).astype(int)
    y_pred_binary = (y_pred > 0.5).astype(int)

    ber = np.mean(y_true_binary != y_pred_binary)
    bler = 1 - np.mean(y_true_binary == y_pred_binary)

    cm = confusion_matrix(y_true_binary, y_pred_binary)
    class_report = classification_report(y_true_binary, y_pred_binary)

    return {
        'BER': ber,
        'BLER': bler,
        'Confusion Matrix': cm,
        'Classification Report': class_report
    }

# Visualization Function for Training/Validation Loss
def plot_training_validation_loss(train_losses, val_losses, title):
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
#########################################################################



# polar_code_generator.py
class PolarCodeGenerator:
    def __init__(self, N, K, crc_type='CRC-7'):
        """
        Polar Code Generator with CRC support

        Args:
            N (int): Total code length
            K (int): Information bit length
            crc_type (str): CRC polynomial type
        """
        self.N = N
        self.K = K
        self.crc_type = crc_type

        # CRC Polynomials
        self.crc_polynomials = {
            'CRC-7': {
                'polynomial': [1, 1, 1, 0, 0, 1, 1],
                'length': 7
            }
        }

    def generate_info_bits(self):
        """
        Generate random information bits

        Returns:
            np.ndarray: Random information bits
        """
        return np.random.randint(2, size=self.K)

    def compute_crc(self, bits):
        """
        Compute CRC checksum using polynomial division

        Args:
            bits (np.ndarray): Input bits

        Returns:
            np.ndarray: CRC checksum bits
        """
        poly_info = self.crc_polynomials.get(self.crc_type)
        if not poly_info:
            raise ValueError(f"Unsupported CRC type: {self.crc_type}")

        polynomial = poly_info['polynomial']
        crc_length = poly_info['length']

        # Convert input to list and pad
        message = bits.tolist() + [0] * crc_length

        # Polynomial long division
        for i in range(len(message) - crc_length):
            if message[i] == 1:
                for j in range(crc_length + 1):
                    message[i + j] ^= polynomial[j] if j < len(polynomial) else 0

        # Return the last 'crc_length' bits
        return np.array(message[-crc_length:], dtype=int)

    def polar_encode(self, info_bits):
        """
        Polar Code Encoding with CRC

        Args:
            info_bits (np.ndarray): Information bits

        Returns:
            np.ndarray: Encoded codeword
        """
        # Append CRC
        crc_bits = self.compute_crc(info_bits)
        extended_info_bits = np.concatenate([info_bits, crc_bits])

        # Basic polar encoding (placeholder)
        codeword = np.zeros(self.N, dtype=int)
        codeword[:len(extended_info_bits)] = extended_info_bits

        return codeword


#part three
######################################################



######################################################



#part 4

# dataset_preparation.py
def prepare_polar_dataset(polar_code_gen, num_samples, snr_db=5, channel_type="AWGN"):
    """
    Prepare dataset for Polar Code simulation

    Args:
        polar_code_gen (PolarCodeGenerator): Polar code generator
        num_samples (int): Number of samples to generate
        snr_db (float): Signal-to-Noise Ratio in dB
        channel_type (str): Channel type

    Returns:
        tuple: Input features and corresponding labels
    """
    # Create channel simulator
    channel_simulator = EnhancedChannelSimulator(channel_type=channel_type)

    # Initialize storage
    X = []
    y = []

    for _ in range(num_samples):
        # Generate information bits
        info_bits = polar_code_gen.generate_info_bits()

        # Encode polar code
        encoded_signal = polar_code_gen.polar_encode(info_bits)

        # Channel simulation
        received_signal = channel_simulator.simulate(encoded_signal, snr_db)

        # Store features and labels
        X.append(received_signal)

        # Binary classification label (based on mean of info_bits)
        y.append(1 if np.mean(info_bits) > 0.5 else 0)

    return np.array(X), np.array(y)

#part 5

# neural_decoder.py





def compute_performance_metrics(y_true, y_pred):
    """
    Compute comprehensive performance metrics

    Args:
        y_true (np.ndarray): True labels
        y_pred (np.ndarray): Predicted labels

    Returns:
        dict: Performance metrics
    """
    # Convert to binary labels if not already
    y_true_binary = (y_true > 0.5).astype(int)
    y_pred_binary = (y_pred > 0.5).astype(int)

    # Bit Error Rate (BER)
    ber = np.mean(y_true_binary != y_pred_binary)

    # Block Error Rate (BLER)
    bler = 1 - np.mean(y_true_binary == y_pred_binary)

    # Confusion Matrix
    cm = confusion_matrix(y_true_binary, y_pred_binary)

    # Classification Report
    class_report = classification_report(y_true_binary, y_pred_binary)

    return {
        'BER': ber,
        'BLER': bler,
        'Confusion Matrix': cm,
        'Classification Report': class_report
    }

def plot_performance_metrics(metrics):
    """
    Create comprehensive performance visualization

    Args:
        metrics (dict): Performance metrics
    """
    # Create a figure with multiple subplots
    fig, axs = plt.subplots(1, 2, figsize=(15, 6))

    # Confusion Matrix Heatmap
    sns.heatmap(
        metrics['Confusion Matrix'],
        annot=True,
        fmt='d',
        cmap='Blues',
        ax=axs[0]
    )
    axs[0].set_title('Confusion Matrix')
    axs[0].set_xlabel('Predicted Label')
    axs[0].set_ylabel('True Label')

    # Performance Metrics Bar Plot
    performance_data = {
        'Bit Error Rate (BER)': metrics['BER'],
        'Block Error Rate (BLER)': metrics['BLER']
    }

    axs[1].bar(performance_data.keys(), performance_data.values())
    axs[1].set_title('Error Rate Performance')
    axs[1].set_ylabel('Error Rate')
    axs[1].set_ylim(0, 1)

    # Add text annotations to bar plot
    for i, (key, value) in enumerate(performance_data.items()):
        axs[1].text(i, value, f'{value:.4f}', ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

####################################################
#Enhanced BER, BLER plot_performance_metrics
class PerformanceVisualizer:
    @staticmethod
    def plot_ber_bler_performance(performance_results, channel_type):
        """
        Create comprehensive BER and BLER performance plots

        Args:
            performance_results (dict): Performance metrics from simulation
            channel_type (str): Channel type (AWGN or Rayleigh)
        """
        # Extract list sizes and SNR range
        list_sizes = performance_results['List Sizes']
        snr_range = performance_results['SNR']

        # Create figure with two subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        fig.suptitle(f'Performance Analysis for {channel_type} Channel')

        # BER Plot with Logarithmic Scale
        ax1.set_title('Bit Error Rate (BER)')
        ax1.set_xlabel('SNR (dB)')
        ax1.set_ylabel('BER')
        ax1.set_yscale('log')  # Logarithmic y-axis
        ax1.set_ylim(1e-4, 1e0)  # Y-axis limit from 10^-4 to 10^0
        ax1.grid(True, which="both", ls="-", alpha=0.5)

        # BLER Plot with Logarithmic Scale
        ax2.set_title('Block Error Rate (BLER)')
        ax2.set_xlabel('SNR (dB)')
        ax2.set_ylabel('BLER')
        ax2.set_yscale('log')  # Logarithmic y-axis
        ax2.set_ylim(1e-4, 1e0)  # Y-axis limit from 10^-4 to 10^0
        ax2.grid(True, which="both", ls="-", alpha=0.5)

        # Color and marker cycles for different list sizes
        colors = ['blue', 'red', 'green']
        markers = ['o', 's', '^']

        # Plot BER for each list size
        for i, list_size in enumerate(list_sizes):
            ber_values = performance_results['BER'][list_size]
            ax1.plot(
                snr_range,
                ber_values,
                label=f'List Size {list_size}',
                color=colors[i],
                marker=markers[i],
                markersize=8
            )

        # Plot BLER for each list size
        for i, list_size in enumerate(list_sizes):
            bler_values = performance_results['BLER'][list_size]
            ax2.plot(
                snr_range,
                bler_values,
                label=f'List Size {list_size}',
                color=colors[i],
                marker=markers[i],
                markersize=8
            )

        # Add legends
        ax1.legend()
        ax2.legend()

        # Adjust layout and display
        plt.tight_layout()
        plt.show()

###################################################
#part 6 main() function




def plot_training_validation_loss(train_losses, val_losses, channel_type, list_size, snr):
    """
    Plot training and validation losses
    """
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title(f'Training and Validation Losses\n{channel_type} Channel, List Size {list_size}, SNR {snr} dB')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()
    plt.show()

def plot_confusion_matrix(y_true, y_pred, channel_type, list_size, snr):
    """
    Plot confusion matrix with enhanced visualization
    """
    # Convert to binary labels
    y_true_binary = (y_true > 0.5).astype(int)
    y_pred_binary = (y_pred > 0.5).astype(int)

    # Compute confusion matrix
    cm = confusion_matrix(y_true_binary, y_pred_binary)

    # Visualization
    plt.figure(figsize=(8, 6))
    sns.heatmap(
        cm,
        annot=True,
        fmt='d',
        cmap='Blues',
        xticklabels=['Negative', 'Positive'],
        yticklabels=['Negative', 'Positive']
    )
    plt.title(f'Confusion Matrix\n{channel_type} Channel, List Size {list_size}, SNR {snr} dB')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.tight_layout()
    plt.show()


#########################################################################################
# Main Simulation Function
def main():
    # Simulation Configuration
    BLOCK_LENGTH = 32
    INFO_BITS = 16
    NUM_SAMPLES = 5000
    EPOCHS = 50
    BATCH_SIZE = 32
    VALIDATION_SPLIT = 0.2

    # Polar Code Generator
    polar_code_gen = PolarCodeGenerator(N=BLOCK_LENGTH, K=INFO_BITS)

    # Prepare dataset
    X, y = prepare_polar_dataset(
        polar_code_gen,
        num_samples=NUM_SAMPLES,
        snr_db=5,  # You can adjust SNR
        channel_type="AWGN"
    )

    # Flatten input features
    X_flattened = X.reshape(X.shape[0], -1)

    # Split dataset
    X_train, X_test, y_train, y_test = train_test_split(
        X_flattened, y,
        test_size=VALIDATION_SPLIT,
        random_state=42
    )

    # Create model and trainer
    input_size = X_train.shape[1]
    model = EnhancedRNNDecoder(input_size)
    trainer = DecoderTrainer(model)

    # Train the model
    train_losses, val_losses = trainer.train(X_train, y_train, epochs=EPOCHS)

    # Plot training and validation losses
    plot_training_validation_loss(
        train_losses,
        val_losses,
        'Training and Validation Losses'
    )

    # Predict on test set
    y_pred = trainer.predict(X_test)

    # Compute and print performance metrics
    metrics = compute_performance_metrics(y_test, y_pred)
    print("Performance Metrics:")
    print(f"BER: {metrics['BER']}")
    print(f"BLER: {metrics['BLER']}")
    print("\nClassification Report:")
    print(metrics['Classification Report'])

    return metrics

# Run the simulation
if __name__ == "__main__":
    main()

#########################################################################################




🚀 Using Device: cuda


ValueError: Target size (torch.Size([32, 1])) must be the same as input size (torch.Size([32]))